# an interactive version of Tensorflow Mandelbrot set tutorial
based on original tutorial at https://www.tensorflow.org/tutorials/mandelbrot

In [1]:
# imports for simulation
import tensorflow as tf
import numpy as np

# imports for visualization
import PIL.Image
from io import BytesIO
from IPython.display import Image, display

# imports for interaction 
from ipywidgets import interact, FloatSlider

In [2]:
def DisplayFractal(a, fmt='jpeg'):
  """Display an array of iteration counts as a
     colorful picture of a fractal."""
  a_cyclic = (6.28*a/20.0).reshape(list(a.shape)+[1])
  img = np.concatenate([10+20*np.cos(a_cyclic),
                        30+50*np.sin(a_cyclic),
                        155-80*np.cos(a_cyclic)], 2)
  img[a==a.max()] = 0
  a = img
  a = np.uint8(np.clip(a, 0, 255))
  f = BytesIO()
  PIL.Image.fromarray(a).save(f, fmt)
  display(Image(data=f.getvalue()))

In [12]:
def recalc(zoom, xoffset, yoffset): 
    print("scale=" + str(zoom) + ", center=[" + str(xoffset) + ":" + str(yoffset) + "]")
    Y, X = np.mgrid[(-2.0/zoom)+yoffset:(2.0/zoom)+yoffset:0.005/zoom, 
                    (-2.0/zoom)+xoffset:(2.0/zoom)+xoffset:0.005/zoom] 
    Z = X+1j*Y
    sess = tf.InteractiveSession()

    xs = tf.constant(Z.astype(np.complex64))
    zs = tf.Variable(xs)
    ns = tf.Variable(tf.zeros_like(xs, tf.float32))

    tf.global_variables_initializer().run()

    # Compute the new values of z: z^2 + x
    zs_ = zs*zs + xs

    # Have we diverged with this new value?
    not_diverged = tf.abs(zs_) < 4

    # Operation to update the zs and the iteration count
    step = tf.group(
      zs.assign(zs_),
      ns.assign_add(tf.cast(not_diverged, tf.float32))
      )

    for i in range(200): step.run()
    DisplayFractal(ns.eval())
    sess.close()

In [13]:
def zoom_changed(zoomer, xoff, yoff):
    zoom = zoomer
    recalc(zoomer, xoff, yoff)
    
interact(zoom_changed, 
         zoomer = FloatSlider(min=1,max=50,step=0.5,value=1, continuous_update=False), 
         xoff = FloatSlider(min=-2, max=2, step=0.05, value=-0.5, continuous_update=False), 
         yoff = FloatSlider(min=-2, max=2, step=0.05, value=0.0, continuous_update=False));

A Jupyter Widget